BYNmasivofy2024*

In [289]:
# Importa las librerias a usar
import pandas as pd
import re
import numpy as np


importar arvhivos a procesar

In [290]:
# ubicacion de los archivos
patch1 = '../input/Kilometros en Servicio MIO - 05-Mar-2024.xlsx'# Se importa el archivo xlxs
patch2 =  '../input/REPORTE FINALIZADO 05_MARZ_2024.xlsx'
patch3 = '../input/JERARQUIAS.xlsx'

In [291]:
# Se importa el archivo xlxs
kmsmio = pd.read_excel(patch1, sheet_name='Programacion')
repofi = pd.read_excel(patch2, sheet_name='Programacion')
bitacora = pd.read_excel(patch2, sheet_name='Bitacora')
jerarquias = pd.read_excel(patch3, sheet_name='Hoja1')
# Se eliminan las filas y las columnas vacias
kmsmio = kmsmio.dropna(axis=0, how='all').dropna(axis=1, how='all')
repofi = repofi.dropna(axis=0, how='all').dropna(axis=1, how='all')
bitacora = bitacora.dropna(axis=0, how='all').dropna(axis=1, how='all')

In [292]:
jerarquias.loc[223]

JERARQUIA HGI          Salida de Operación::Por Operador::No Relevo
JERARQUIA HGI RECO     Salida de Operación::Por Operador::No Relevo
JERARQUIA HGI RECO2    Salida de Operación::Por Operador::No Relevo
CALIFICACION                                                      2
Name: 223, dtype: object

In [293]:
# Se crea el df de adicionales
adi = kmsmio[kmsmio['Kilometros Programados Plan'].isna()]

Porcesar K2

In [294]:
# Filtra el DF por Kms en Servicio
kmsmio = kmsmio[kmsmio['Tipo Kilómetros']== 'Kms en Servicio']
kmsmio = kmsmio[kmsmio['Kilometros Programados Plan'].notna()]

In [295]:
# Cambiamos la hora 24 por 0
kmsmio['Desde'] = kmsmio['Desde'].replace('24', '0')
kmsmio = kmsmio[kmsmio['Desde'] != '0']


In [296]:
# Se crea una nueva columna 'id'
kmsmio['id'] = kmsmio.apply(lambda row: '-'.join([str(row['Línea']), str(row['Número de tarea vehículo']), 
                                                   str(row['Desde']), str(row['hasta'])]), axis=1)


In [297]:
# Se verifica el tamaño del df
kmsmio.shape

(18645, 40)

In [298]:
# Se verifica que la columna id no tenga id repetidos
kmsmio['id'].value_counts().sort_values(ascending=False)

id
A14A-1-05:00:00-05:12:10                              1
T47B-2-19:21:48-20:01:10                              1
A14A-1-05:26:00-05:38:10                              1
A14A-1-05:38:55-05:51:15                              1
A14A-1-05:52:00-06:04:43                              1
                                                     ..
A35A-99753-1900-01-01 00:03:04-1900-01-01 00:14:39    1
A35A-99753-23:03:04-23:14:39                          1
A35A-99753-21:33:04-21:44:07                          1
A35A-99753-21:30:02-21:30:17                          1
A35A-99753-1900-01-01 00:16:10-1900-01-01 00:29:23    1
Name: count, Length: 18645, dtype: int64

Procesar reporte finalizado

In [299]:
# Filtra el DF por Kms en Servicio
repofi = repofi[repofi['Tipo de kilometros']== 'Kms en Servicio']
# Se crea una nueva columna 'id'
repofi['id'] = repofi.apply(lambda row: '-'.join([str(row['Línea']), str(row['Número de tarea vehículo']), 
                                                  str(row['desde']),str(row['hasta'])]), axis=1)


In [300]:
# Se verifica que el id no tenga datos repetidos
repofi['id'].value_counts().sort_values(ascending=False)

id
A14A-1-05:00:00-05:12:10                              1
T47B-2-19:21:48-20:01:10                              1
A14A-1-05:26:00-05:38:10                              1
A14A-1-05:38:55-05:51:15                              1
A14A-1-05:52:00-06:04:43                              1
                                                     ..
A35A-99753-1900-01-01 00:03:04-1900-01-01 00:14:39    1
A35A-99753-23:03:04-23:14:39                          1
A35A-99753-21:33:04-21:44:07                          1
A35A-99753-21:30:02-21:30:17                          1
A35A-99753-1900-01-01 00:16:10-1900-01-01 00:29:23    1
Name: count, Length: 18645, dtype: int64

In [301]:
# Se verifica el tamaño del df
repofi.shape

(18645, 32)

In [302]:
reporf = repofi[['id', 'Largo', 'Número de secuencia de carro', 'Secuencia de arcos', 'Concesionario de Transporte']]

Se realiza el respectivo Merge con los dos df

In [303]:
# Se realiza el merge con outer en la columna id
merge = pd.merge(kmsmio, repofi, on='id', how='outer')

In [304]:
# Se verifica que el numero de filas en los df sea igual (True), para poder concluir que el merge fue exitoso
merge.shape[0] == kmsmio.shape[0] == repofi.shape[0]

True

Calificaciones en Bitacora

In [305]:
# Selecciona las columnas a usar
jerarquias = jerarquias[['JERARQUIA HGI RECO2',	'CALIFICACION']]

In [306]:
# Esto elimina los espacios en blanco adelante y atras
jerarquias['JERARQUIA HGI RECO2'] = jerarquias['JERARQUIA HGI RECO2'].str.strip()

In [307]:
#combina los el df de bitacora con el de jerarquias
bitacora = pd.merge(bitacora, jerarquias, left_on='Jerarquía', right_on='JERARQUIA HGI RECO2', how='left' )

In [308]:
# Esta linea es para verificar las filas que no se cruzaron por jerarquia y agregarlas a la matriz fe jererquia en input
x = bitacora[(bitacora['Jerarquía'].str.startswith('Salida'))& bitacora['CALIFICACION'].isnull()]
x['Jerarquía'].value_counts()

Series([], Name: count, dtype: int64)

Merge con bitacora


In [309]:
# Se reemplan las filas nulas por ceros de las dos columnas para posteriormente carbiar el tipo de dato
bitacora[['Tarea', 'Viaje']] = bitacora[['Tarea', 'Viaje']].fillna(0).astype(int)

In [310]:
bitacora

,# Caso MC,Tipo novedad,Jerarquía,Tarea,Viaje,Parada,Operador,Línea,Vehículo,Hora,...,Descripción,Jerarquia,Tipo de Novedad,Novedad,Retoma/Salida,Unnamed: 16,Unnamed: 17,Unnamed: 18,JERARQUIA HGI RECO2,CALIFICACION
0,I20051440,Incumplimiento del COT,Incumplimiento del COT::Flota::Motor,176,2,TAGUAC7,22383.0,A44A,22131.0,04:30,...,Caso: I20051440\nHora: 04:30\nLinea: A44A\nTar...,Incumplimiento del COT::Flota::Motor,Incumplimiento del COT,Motor,NaN,176-2,I20051440,NaN,NaN,NaN
1,I20051435,Incumplimiento del COT,Incumplimiento del COT::Disponibilidad De Oper...,756,1,UNIVCOM9,22721.0,A13D,23045.0,04:37,...,Caso: I20051435\nHora: 04:37\nLinea: A13D\nTar...,Incumplimiento del COT::Disponibilidad De Oper...,Incumplimiento del COT,Disponibilidad De Operador,NaN,756-1,I20051435,NaN,NaN,NaN
2,I20051439,Incumplimiento del COT,Incumplimiento del COT::Flota::Falla Eléctrica,113,5,A.SANI16,22728.0,A44B,23011.0,04:45,...,Caso: I20051439\nHora: 04:45\nLinea: A44B\nTar...,Incumplimiento del COT::Flota::Falla Eléctrica,Incumplimiento del COT,Falla Eléctrica,NaN,113-5,I20051439,NaN,NaN,NaN
3,2374276,Incidente,Incidente::Por Flota::Con Protocolo Tecnologic...,61,3,T103D26I,22735.0,A44B,23051.0,04:47,...,Caso: 2374276\nHora: 04:47\nLinea: A44B\nTarea...,Incidente::Por Flota::Con Protocolo Tecnologic...,Incidente,Ibis,NaN,61-3,2374276,NaN,NaN,NaN
4,20022917,Retoma,Retoma,176,5,A.SANI17,22383.0,A44A,22065.0,04:58,...,Caso: 20022917\nHora: 04:58\nLinea: A44A\nTare...,Retoma,Retoma,Retoma,No hay salida,176-5,20022917,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,NaN,No retoma,No retoma,99635,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Por tiempo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
531,NaN,No retoma,No retoma,99046,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Tarea adicional a la operación,NaN,NaN,NaN,NaN,NaN,NaN,NaN
532,NaN,No retoma,No retoma,884,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Por tiempo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
533,NaN,No retoma,No retoma,99051,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Por tiempo,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [311]:
# Se selecionan las columnas de Bitacora que se van a usar
bitacora = bitacora[['Jerarquía',	'Tarea',	'Viaje', 'Parada', 'Vehículo', 'Novedad', 'CALIFICACION']]
# Se filtran los casos que no tiene numero de viaje ya que no tiene retoma
bitacora = bitacora[bitacora['Viaje'] != 0]

In [312]:
# Se compara las filas de los df, el resultado debe ser False
bitacora.shape[0]==bitacora.shape[0]

True

In [313]:
bitacora

,Jerarquía,Tarea,Viaje,Parada,Vehículo,Novedad,CALIFICACION
0,Incumplimiento del COT::Flota::Motor,176,2,TAGUAC7,22131.0,Motor,NaN
1,Incumplimiento del COT::Disponibilidad De Oper...,756,1,UNIVCOM9,23045.0,Disponibilidad De Operador,NaN
2,Incumplimiento del COT::Flota::Falla Eléctrica,113,5,A.SANI16,23011.0,Falla Eléctrica,NaN
3,Incidente::Por Flota::Con Protocolo Tecnologic...,61,3,T103D26I,23051.0,Ibis,NaN
4,Retoma,176,5,A.SANI17,22065.0,Retoma,NaN
...,...,...,...,...,...,...,...
474,Salida de Operación::MC::Cambio De Linea,884,237,UNIVCOM7,23024.0,Cambio De Linea,2.0
475,Incidente::Por Operador::No Usa O Usa Mal El P...,154,102,UNIVCOM5,23041.0,No Usa O Usa Mal El Protocolo De Comunicacion,NaN
476,Salida de Operación::Por Operador::Abandono De...,154,102,UNIVCOM5,23041.0,Injustificado,2.0
477,Salida de Operación::Por Accidente::Golpe con ...,99051,139,C126K28E,53023.0,Golpe con tercero,2.0


In [314]:
# Se agrega al merge anterior la informacion de bitacora
# Importante tener en cuenta los vehiculos que tengan mas de un caso en la bitacora, esto genera id duplicados.
mergebitacora = pd.merge(merge, bitacora, left_on=['Número de tarea vehículo_x', 'Número de viaje_x'], right_on=['Tarea', 'Viaje'], how='outer' )

In [315]:
# Revisar este resultado, ya que los valore sque no coinceden deven se iguales en el filtro
mergebitacora.shape[0]- merge.shape[0]

34

In [316]:
# Cambia Blanco y Negro Masivo por BYN en la columna Concesionario Programado
mergebitacora['Concesionario Programado'] = mergebitacora['Concesionario Programado'].replace('Blanco y Negro Masivo','BYN')

In [317]:
# Reemplaza la calificacion de RF a la de k2 por calificaciones menores si es el caso
def incumplidos_y_modificar(row):
    # Verifica si tanto 'Incumplidos_x' como 'Incumplidos_y' no son nulos
    if pd.notnull(row['Incumplidos_x']) and pd.notnull(row['Incumplidos_y']):
        # Si ambos valores no son nulos, devuelve el mínimo entre ellos
        return min(row['Incumplidos_x'], row['Incumplidos_y'])
    # Si 'Incumplidos_x' no es nulo pero 'Incumplidos_y' sí lo es
    elif pd.notnull(row['Incumplidos_x']):
        # Devuelve el valor de 'Incumplidos_x'
        return row['Incumplidos_x']
    elif pd.isnull(row['Incumplidos_x']):
    # Si 'Incumplidos_x' es nulo
        return row['Incumplidos_x']
    else:
        # Devuelve el valor de 'Incumplidos_y'
        return row['Incumplidos_y']

In [318]:
# Aplica la funcion que reemplaza los valores en la columna imcumplidos_y
mergebitacora['Incumplidos_y'] = mergebitacora.apply(incumplidos_y_modificar, axis=1)


In [319]:
x = mergebitacora

In [320]:
x = x[['id','Incumplidos_x','Incumplidos_y', 'CALIFICACION']]

In [321]:
x[(x['Incumplidos_x'].isna()) & (x['Incumplidos_y'].notnull()) ]

,id,Incumplidos_x,Incumplidos_y,CALIFICACION


In [322]:
# Revisar 
x[x['Incumplidos_x']> x['Incumplidos_y']]

,id,Incumplidos_x,Incumplidos_y,CALIFICACION
1159,P21A-42-15:42:40-16:53:50,3.0,2.0,NaN
1690,A44B-61-05:07:15-05:37:54,3.0,2.0,NaN
7450,A02-292-12:27:43-12:51:34,3.0,2.0,NaN
7451,A02-292-12:52:31-13:19:14,3.0,2.0,NaN
8289,E21-337-07:35:06-07:36:06,3.0,2.0,NaN
8723,P27D-357-07:04:40-07:05:40,3.0,2.0,NaN
9882,P27D-417-06:35:17-06:36:17,3.0,2.0,NaN
10115,A21-428-09:09:58-09:28:43,3.0,2.0,NaN
10116,A21-428-09:29:22-09:53:24,3.0,2.0,NaN
10167,A21-428-09:53:39-09:54:39,3.0,2.0,NaN


In [323]:
x[x['Incumplidos_x']< x['Incumplidos_y'] ]

,id,Incumplidos_x,Incumplidos_y,CALIFICACION


In [324]:
# Definir una función para aplicar a cada grupo
def eliminar_duplicados_bitacora(row):
    # Verificar si hay al menos un valor en 'Incumplidos_x'
    if row['Incumplidos_x'].isnull() and row['Incumplidos_x'].isnull():



        
        return row[row['Incumplidos_x'].notnull()]  # Conservar la fila con 'Incumplidos_x' no nulo
    # Verificar si hay al menos un valor en 'Incumplidos_y'
    elif row['Incumplidos_y'].notnull().any():
        return row[row['Incumplidos_y'].notnull()]  # Conservar la fila con 'Incumplidos_y' no nulo
    # Conservar la fila con el valor más bajo en 'CALIFICACION'
    else:
        return row.loc[row['CALIFICACION'].idxmin()]

In [152]:
# Muestra los id duplicados producto del merge con bitacora
mergebitacora[mergebitacora['id'].duplicated()]

,Fecha_x,Día tipo_x,Designación de tarea vehículo_x,Línea_x,Tipo de viaje corto_x,Sentido_x,Tipo Kilómetros,Número de Vehículo,Concesionario de Transporte_x,Número de tarea vehículo_x,...,Kms Planeados,Kms Perdidos,Kms cancelados,Jerarquía,Tarea,Viaje,Parada,Vehículo_y,Novedad,CALIFICACION
326,2024-03-05,DHABIL,13,A01A,Viaje comercial,Ida,Kms en Servicio,54013,BYNCPC,13,...,4704,NaN,NaN,Salida de Operación::Por Flota::Motor,13.0,21.0,SANBO2,54013.0,Motor,2.0
332,2024-03-05,DHABIL,13,A01A,Viaje comercial,Ida,Kms en Servicio,54013,BYNCPC,13,...,4704,4166.0,NaN,Salida de Operación::Por Flota::Motor,13.0,26.0,SANPAS6,54013.0,Motor,2.0
334,2024-03-05,DHABIL,13,A01A,Viaje comercial,Ida,Kms en Servicio,NaN,NaN,13,...,4704,4704.0,NaN,Retoma,13.0,27.0,SANBO2,22117.0,Retoma,NaN
379,2024-03-05,DHABIL,14,A32,Viaje comercial,Ida,Kms en Servicio,NaN,NaN,14,...,4430,NaN,NaN,Retoma,14.0,48.0,C62K1BIS,23033.0,Retoma,NaN
1158,2024-03-05,DHABIL,42,P21A,Viaje comercial,Ida,Kms en Servicio,22142,BYN,42,...,14534,NaN,NaN,Salida de Operación::UTRYT::IBIS,42.0,109.0,UNIVCOM1,22142.0,IBIS,1.0
1689,2024-03-05,DHABIL,61,A44B,Viaje comercial,Ida,Kms en Servicio,23051,BYN,61,...,7651,NaN,NaN,Salida de Operación::UTRYT::IBIS,61.0,3.0,TAGUAC7,23051.0,IBIS,1.0
2991,2024-03-05,DHABIL,106,E21,Viaje comercial,Ida,Kms en Servicio,NaN,NaN,106,...,16401,NaN,NaN,Salida de Operación::Por Flota::Motor,106.0,233.0,UNIV1,21053.0,Motor,2.0
4232,2024-03-05,DHABIL,154,A17D,Viaje comercial,Ida,Kms en Servicio,23041,BYN,154,...,8840,NaN,NaN,Salida de Operación::Por Operador::Abandono De...,154.0,102.0,UNIVCOM5,23041.0,Injustificado,2.0
4852,2024-03-05,DHABIL,176,A44A,Viaje comercial,Ida,Kms en Servicio,22029,BYN,176,...,6112,518.0,NaN,Incidente::Despacho::Asignacion Incorrecta De ...,176.0,153.0,K29C80,22029.0,Asignacion Incorrecta De Tarea En Ivu Vehicle,NaN
4853,2024-03-05,DHABIL,176,A44A,Viaje comercial,Ida,Kms en Servicio,22029,BYN,176,...,6112,518.0,NaN,Salida de Operación::Por Operador::Abandono De...,176.0,153.0,TAGUAC7,22029.0,NO RELEVO,2.0


Revision de Adicionales

Columna condicional 

In [120]:
# Se cambian los vehiculos del cierre por los de k2##
mergebitacora['Número de secuencia de carro'] = [x if pd.notna(x) else y for x, y in zip(mergebitacora['Número de secuencia de carro'], mergebitacora['Número de Vehículo'])]
# Marca en la columnas Cambios los cambios que realiza el código
mergebitacora['condicion'] = [ np.nan if  pd.notna(x) else 'Se Cambia # V RF' for x, y in zip(mergebitacora['Número de secuencia de carro'], mergebitacora['Número de Vehículo'])]


In [121]:
# Función para actualizar los valores de Largo_y si Largo_x es mayor
def actualizar_largo(row):
    if pd.isna(row['Largo_y']):
        return row['Largo_x']
    elif row['Largo_x'] > row['Largo_y']:
        return row['Largo_x']
    else:
        return row['Largo_y']

In [122]:
# # Función para actualizar el texto de la columna condicion para Largo_y si Largo_x es mayor
def actualizar_largo_texto(row):
    if pd.isna(row['Largo_y']) and pd.isna(row['condicion']) :
        return 'Cambia largo por NAN'
    elif pd.isna(row['Largo_y']) and pd.notnull(row['condicion']) :
        return row['condicion'] + '/Cambia largo por NAN'
    elif row['Largo_x'] > row['Largo_y'] and pd.isna(row['condicion']):
        return 'Cambia x Mayor largo'
    elif row['Largo_x'] > row['Largo_y'] and pd.notnull(row['condicion']) :
        return row['condicion'] + '/Cambia x Mayor largo'
    else:
        if pd.notnull(row['condicion']):
            return row['condicion']
            


In [123]:
# Marcar las filas donde se realizan cambios en una nueva columna
mergebitacora['condicion'] = mergebitacora.apply(actualizar_largo_texto, axis=1)

In [124]:
# Aplicar la función a la columna Largo_y
mergebitacora['Largo_y'] = mergebitacora.apply(actualizar_largo, axis=1)

In [125]:
#Se verifican los cambios realizados
mergebitacora['condicion'].value_counts()

condicion
Se Cambia # V RF                         2332
Cambia x Mayor largo                      363
Se Cambia # V RF/Cambia x Mayor largo      20
Name: count, dtype: int64

In [126]:
'Número de secuencia de carro', 'Secuencia de arcos', 'Concesionario Programado'

('Número de secuencia de carro',
 'Secuencia de arcos',
 'Concesionario Programado')

In [127]:
# Condiciones para la columna condicional
condicion1 = mergebitacora['Largo_x'] != mergebitacora['Largo_y']
condicion2 = mergebitacora['Número de Vehículo'] != mergebitacora['Número de secuencia de carro']
condicion3 = mergebitacora['Secuencia de arcos_x'] != mergebitacora['Secuencia de arcos_y']
condicion4 = mergebitacora['Concesionario de Transporte_x'] != mergebitacora['Concesionario Programado']

# Aplicar condiciones y asignar valores a la nueva columna
mergebitacora['condicional2'] = np.where(condicion1 & condicion2 & condicion3 & condicion4, 'Revisar largo, parada, vehículo y consecionario',
                                    np.where(condicion1 & condicion2 & condicion3, 'Revisar largo, parada y vehículo',
                                        np.where(condicion1 & condicion2 & condicion4, 'Revisar largo, parada y consecionario',         
                                            np.where(condicion1 & condicion3 & condicion4, 'Revisar largo, parada y consecionarios',
                                                np.where(condicion2 & condicion3 & condicion4, 'Revisar vehiculo, parada y consecionarios',        
                                                    np.where(condicion1 & condicion2, 'Revisar largo y vehiculo',
                                                        np.where(condicion1 & condicion3, 'Revisar largo y parada',
                                                            np.where(condicion1 & condicion4, 'Revisar largo y consecionario', 
                                                                np.where(condicion2 & condicion3, 'Revisar vehiculo y parada',
                                                                    np.where(condicion2 & condicion4, 'Revisar vehiculo y consecionario',        
                                                                        np.where(condicion3 & condicion4, 'Revisar parada y consecionario',                                            
                                                                            np.where(condicion1, 'Revisar largo', 
                                                                                np.where(condicion2, 'Revisar numero de vehiculo',
                                                                                    np.where(condicion3, 'Revisar paradas',
                                                                                        np.where(condicion4, 'Revisar consesionario', 'OK')))))))))))))))

In [128]:
mergebitacora['condicional2'].value_counts()

condicional2
OK                                       9946
Revisar consesionario                    5540
Revisar vehiculo y consecionario         2603
Revisar largo y consecionario             243
Revisar largo                             236
Revisar numero de vehiculo                100
Revisar largo y vehiculo                    7
Revisar largo, parada y consecionario       4
Name: count, dtype: int64

In [129]:
# Exporta a un archivo excel el Df
mergebitacora.to_excel('../output/prueba_mergebita.xlsx', index=False, sheet_name='k2_procesado')